# Jonathan Halverson
# Thursday, April 28, 2016
# Notes on Spark

Spark is a big data processing engine with many advantages over Hadoop MapReduce. For instance, it is fast, has rich APIs and can be used for interactive SQL queries, streaming, machine learning and graph processing.

Parallel data analysis has become commonplace. Spark offers three main benefits over MapReduce: (1) It is easy to use allowing one to create applications on a laptop using a high level API, (2) Spark is fast thanks to in-memory computing and (3) it is a general engine allowing for different tasks whereas Hadoop has an entire ecosystem of heterogeneous engines.

### Chapter 1

Spark is a cluster computing platform designed to be fast and general-purpose. It is highly accessible offering APIs in Java, Scala and Python. Spark can access any Hadoop data source like Cassandra. Spark comes with a unified stack which means improvements to the core benefit all. It is also easier to maintain.

Spark Core contains components for scheduling, memory manangement, fault recovery and interacting with storage systems.

Spark SQL can be used to work with structured data. Hive Query Language or HQL can also be used. One may mix Spark SQL with various data manipulations. Spark SQL is a replacement to Shark.

Spark Streaming allows for the processing of live data streams. The streaming API closely matches that of the RDD API.

GraphX a is library for manipulating graphs like a social network graph. GraphX extends the Spark RDD API.

A cluster manager is needed to run Spark. It was designed to scale efficiently from one to thousands of compute nodes. Cluster manager choices are Hadoop YARN, Apache Mesos and the Standalone Scheduler.

Data wrangling is transforming data into a particular format so that it can be analyzed. Spark offers support to call out to external programs like Matlab and R.

Software engineers can wrote code to process data and not worry about how the data is stored or about how the analysis is parallelized. They have the ability to monitor, inspect and tune their applications.

The store layers for Spark include any file in HDFS, Amazon S3, Cassandra, Hive and HBase.

### Chapter 2

Spark is written in Scala and runs on the Java Virtual Machine. Why not write it in C++ for the speed advantages? Spark requires Java 6 or higher. Java 8 supports lambda expressions. On a laptop, Spark runs in local mode.

Spark provides different shells for interactive use. Parallel computations can be carried out from the shell on distributed clusters.

In Spark, we express our computation through operations on distributed collections that are automatically parallelized across the cluster. These collections are resilient distributed datasets or RDDS. The driver program was the Spark shell for the intro example. A SparkContest represents a connection to a computing cluster. In the shell the variable sc is automatically created and it is a SparkContext. A SparkContext can be used to build RDDs which can be data from some source (textFile) or data generated in the shell (parallelize).

Lauch the Python shell with: pyspark